<a href="https://colab.research.google.com/github/poeala/ads1-notebooks/blob/master/ProyectoParteIII%2BAndrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este trabajo, desarrollé un notebook con el objetivo de evaluar la posibilidad de predecir si un pokemón es legendario o no de acuerdo a sus stats. Para eso apliqué técnicas seleccioné las columnas que me parecieron más adecuadas y eliminé las que consideré innecesarias para el modelo, entrené un modelo de clasificación de tipo árbol de decisión como el desarrollado en clases y evalué su desempeño utilizando métricas estándar.
En la primera parte, observé los datos y los describí para decidir cuales características eran más adecuadas, evalué y eliminé las filas con campos nulos y las columnas con datos irrelevantes. A continuación, seleccioné el algoritmo de clasificación con el que más me familiaricé y consideré adecuado para el problema en cuestión, luego entrené el modelo con las características seleccionadas. Posteriormente, calculé las métricas clave aprendidas en clase como precisión, recall, f1-score y exactitud para evaluar la capacidad predictiva del modelo. Finalmente, saqué conclusiones basadas en los resultados obtenidos:
 Se observa que en el entrenamiento hay una elevada precisión, ya que el modelo acertó en el 100% de tanto legendarios como no legendarios, esto puede indicar sobreajuste de datos, ya que el modelo memorizo los resultados, es posible que haya eliminado muchas filas de mi DataSet.
 En el testing el modelo identificó muy bien a los pokemón no legendarios, además detectó a todos los poklemón legendarios, pero predijo que un pokemón era legendario cuando no lo era, por lo que la precisión de legendario es de 67% (baja), sospecho que la precisión es baja debido al sobreajustes de datos, hay muy pocos valores para el testeo, ya que había solo 2 pokemón legendartios en la parte de testeo.

In [103]:
!pip install gdown

In [104]:
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [105]:
file_id = '1sYWgZoorF-u4e0oL3BMGnoGpXV38uTiI'
output = 'pokemon.csv'
gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=False)
df = pd.read_csv(output)

Downloading...
From: https://drive.google.com/uc?id=1sYWgZoorF-u4e0oL3BMGnoGpXV38uTiI
To: /content/pokemon.csv
100%|██████████| 162k/162k [00:00<00:00, 62.9MB/s]


In [106]:
df.head()

,name,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,Bulbasaur,['Overgrow'. 'Chlorophyll'],1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,88.1,1,65,65,45,grass,poison,6.9,1,False
1,Ivysaur,['Overgrow'. 'Chlorophyll'],1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,88.1,2,80,80,60,grass,poison,13.0,1,False
2,Venusaur,['Overgrow'. 'Chlorophyll'],1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,...,88.1,3,122,120,80,grass,poison,100.0,1,False
3,Charmander,['Blaze'. 'Solar Power'],0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,False
4,Charmeleon,['Blaze'. 'Solar Power'],0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,False


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               801 non-null    object 
 1   abilities          801 non-null    object 
 2   against_bug        801 non-null    float64
 3   against_dark       801 non-null    float64
 4   against_dragon     801 non-null    float64
 5   against_electric   801 non-null    float64
 6   against_fairy      801 non-null    float64
 7   against_fight      801 non-null    float64
 8   against_fire       801 non-null    float64
 9   against_flying     801 non-null    float64
 10  against_ghost      801 non-null    float64
 11  against_grass      801 non-null    float64
 12  against_ground     801 non-null    float64
 13  against_ice        801 non-null    float64
 14  against_normal     801 non-null    float64
 15  against_poison     801 non-null    float64
 16  against_psychic    801 non

In [108]:
df.describe()

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,experience_growth,height_m,hp,percentage_male,pokedex_number,sp_attack,sp_defense,speed,weight_kg,generation
count,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,801.000000,...,8.010000e+02,781.000000,801.000000,703.000000,801.000000,801.000000,801.000000,801.000000,781.000000,801.000000
mean,0.996255,1.057116,0.968789,1.073970,1.068976,1.065543,1.135456,1.192884,0.985019,1.034020,...,1.054996e+06,1.163892,68.958801,55.155761,401.000000,71.305868,70.911361,66.334582,61.378105,3.690387
std,0.597248,0.438142,0.353058,0.654962,0.522167,0.717251,0.691853,0.604488,0.558256,0.788896,...,1.602558e+05,1.080326,26.576015,20.261623,231.373075,32.353826,27.942501,28.907662,109.354766,1.930420
min,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.250000,0.250000,0.000000,0.250000,...,6.000000e+05,0.100000,1.000000,0.000000,1.000000,10.000000,20.000000,5.000000,0.100000,1.000000
25%,0.500000,1.000000,1.000000,0.500000,1.000000,0.500000,0.500000,1.000000,1.000000,0.500000,...,1.000000e+06,0.600000,50.000000,50.000000,201.000000,45.000000,50.000000,45.000000,9.000000,2.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000e+06,1.000000,65.000000,50.000000,401.000000,65.000000,66.000000,65.000000,27.300000,4.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.059860e+06,1.500000,80.000000,50.000000,601.000000,91.000000,90.000000,85.000000,64.800000,5.000000
max,4.000000,4.000000,2.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,1.640000e+06,14.500000,255.000000,100.000000,801.000000,194.000000,230.000000,180.000000,999.900000,7.000000


In [109]:
df.isna().sum()/df.shape[0]*100

,0
name,0.000000
abilities,0.000000
against_bug,0.000000
against_dark,0.000000
against_dragon,0.000000
against_electric,0.000000
against_fairy,0.000000
against_fight,0.000000
against_fire,0.000000
against_flying,0.000000


In [110]:
to_drop = ['name', 'classfication', 'japanese_name', 'pokedex_number', 'abilities']

In [111]:
df = df.drop(columns=to_drop)

In [112]:
df.isna().sum()/df.shape[0]*100

,0
against_bug,0.000000
against_dark,0.000000
against_dragon,0.000000
against_electric,0.000000
against_fairy,0.000000
against_fight,0.000000
against_fire,0.000000
against_flying,0.000000
against_ghost,0.000000
against_grass,0.000000


In [113]:
df.dropna(inplace=True)

In [114]:
df.isna().sum()/df.shape[0]*100

,0
against_bug,0.0
against_dark,0.0
against_dragon,0.0
against_electric,0.0
against_fairy,0.0
against_fight,0.0
against_fire,0.0
against_flying,0.0
against_ghost,0.0
against_grass,0.0


In [115]:
df.shape

(339, 36)

In [116]:
df['is_legendary'] = df['is_legendary'].map({True: 'Yes', False: 'No'})

In [117]:
X = df.drop("is_legendary", axis=1).copy()
y = df.is_legendary.copy()

In [118]:
X

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation
0,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,0.25,...,0.7,45,88.1,65,65,45,grass,poison,6.9,1
1,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,0.25,...,1.0,60,88.1,80,80,60,grass,poison,13.0,1
2,1.00,1.0,1.0,0.5,0.5,0.50,2.0,2.0,1.0,0.25,...,2.0,80,88.1,122,120,80,grass,poison,100.0,1
5,0.25,1.0,1.0,2.0,0.5,0.50,0.5,1.0,1.0,0.25,...,1.7,78,88.1,159,115,100,fire,flying,90.5,1
11,0.50,1.0,1.0,2.0,1.0,0.25,2.0,2.0,1.0,0.25,...,1.1,60,50.0,90,80,70,bug,flying,32.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,0.25,1.0,0.0,1.0,1.0,0.00,1.0,1.0,2.0,1.00,...,0.2,55,50.0,50,105,96,ghost,fairy,0.7,7
778,2.00,2.0,1.0,2.0,1.0,0.50,0.5,1.0,2.0,2.00,...,0.9,68,50.0,70,70,92,water,psychic,19.0,7
779,1.00,1.0,2.0,0.5,2.0,2.00,0.5,1.0,0.0,0.50,...,3.0,78,50.0,135,91,36,normal,dragon,185.0,7
782,0.50,0.5,2.0,0.5,4.0,1.00,0.5,2.0,1.0,0.50,...,1.2,55,50.0,65,70,65,dragon,fighting,47.0,7


In [119]:
y.unique()

array(['No', 'Yes'], dtype=object)

In [120]:
df['is_legendary'].value_counts()

,count
is_legendary,
No,334
Yes,5


In [121]:
df['is_legendary'].unique()

array(['No', 'Yes'], dtype=object)

In [122]:
y.value_counts(normalize=True)*100

,proportion
is_legendary,
No,98.525074
Yes,1.474926


In [123]:
from sklearn.model_selection import train_test_split

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [125]:
X_train

,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,against_grass,...,height_m,hp,percentage_male,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation
363,1.00,1.0,1.0,2.0,1.0,2.00,1.0,1.0,1.0,2.00,...,1.1,90,50.0,75,70,45,ice,water,87.6,3
635,0.50,1.0,1.0,1.0,0.5,0.50,1.0,2.0,1.0,0.25,...,1.1,55,50.0,50,55,60,bug,fire,28.8,5
86,1.00,1.0,1.0,2.0,1.0,2.00,1.0,1.0,1.0,2.00,...,1.7,90,50.0,70,95,70,water,ice,120.0,1
271,2.00,1.0,1.0,1.0,1.0,1.00,1.0,2.0,1.0,1.00,...,1.5,80,50.0,90,100,70,water,grass,55.0,3
685,4.00,1.0,1.0,1.0,2.0,1.00,1.0,1.0,1.0,1.00,...,0.4,53,50.0,37,46,45,dark,psychic,3.5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,1.00,1.0,0.0,1.0,1.0,0.25,1.0,1.0,2.0,1.00,...,1.3,40,50.0,100,120,90,psychic,fairy,54.5,1
700,0.25,0.5,1.0,2.0,2.0,0.50,1.0,2.0,1.0,0.50,...,0.8,78,50.0,74,63,118,fighting,flying,21.5,6
415,0.50,1.0,1.0,2.0,1.0,0.25,2.0,2.0,1.0,0.25,...,1.2,70,0.0,80,102,40,bug,flying,38.5,4
597,1.00,1.0,0.5,0.5,0.5,2.00,4.0,1.0,1.0,0.25,...,1.0,74,50.0,54,116,20,grass,steel,110.0,5


In [126]:
from sklearn.preprocessing import OneHotEncoder

In [127]:
ohe = OneHotEncoder(sparse_output=False).fit(X_train[["type1","type2"]])

In [128]:
encoded = ohe.transform(X_train[["type1","type2"]])

In [129]:
encoded_df = pd.DataFrame(columns=ohe.get_feature_names_out(), data=encoded, index=X_train.index)

In [130]:
X_train = pd.concat([X_train, encoded_df], axis="columns")

In [131]:
encoded_test = ohe.transform(X_test[["type1","type2"]])
encoded_df_test = pd.DataFrame(columns=ohe.get_feature_names_out(), data=encoded_test, index=X_test.index)

In [132]:
X_test = pd.concat([X_test, encoded_df_test], axis="columns")

In [133]:
X_train = X_train.drop(["type1","type2"], axis=1)
X_test = X_test.drop(["type1","type2"], axis=1)

In [134]:
reglas = {"Yes": 1, "No": 0}

y_train = y_train.map(reglas)
y_test = y_test.map(reglas)

In [135]:
df['is_legendary'].unique()

array(['No', 'Yes'], dtype=object)

In [136]:
y_train

,is_legendary
363,0
635,0
86,0
271,0
685,0
...,...
121,0
700,0
415,0
597,0


In [137]:
y_test

,is_legendary
138,0
259,0
674,0
169,0
519,0
...,...
721,0
629,0
654,0
412,0


In [150]:
from sklearn.tree import DecisionTreeClassifier

In [152]:
clf = DecisionTreeClassifier(max_depth=10, random_state=42).fit(X_train, y_train)

In [153]:
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

In [154]:
train_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [155]:
from sklearn.metrics import classification_report

In [156]:
print("Training")
print(classification_report(y_train, train_pred))
print("-------------------------------------------")
print("Testing")
print(classification_report(y_test, test_pred))

Training
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00         3

    accuracy                           1.00       237
   macro avg       1.00      1.00      1.00       237
weighted avg       1.00      1.00      1.00       237

-------------------------------------------
Testing
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       100
           1       0.67      1.00      0.80         2

    accuracy                           0.99       102
   macro avg       0.83      0.99      0.90       102
weighted avg       0.99      0.99      0.99       102

